In [1]:
# @title
import ipywidgets as widgets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from sklearn import preprocessing
import numpy as np
import sys
import os

#####################
# MONTAGEM DO DRIVE #
#####################
from google.colab import drive
if not os.path.ismount('/content/drive'):
    print("Montando o Google Drive...")
    drive.mount('/content/drive')

###########################
# CARREGAMENTO DE MÓDULOS #
###########################

# Caminho dos módulos do projeto armazenados no Google Drive
caminho_modulos = '/content/drive/MyDrive/Colab Notebooks/VC/Extracao_Classificacao/modulos'
from pydoc import importfile
# Importação de módulos Python
sift_module = importfile(caminho_modulos+'/descritores/sift.py')
histograma_module = importfile(caminho_modulos+'/descritores/histograma.py')
lbp_module = importfile(caminho_modulos+'/descritores/lbp.py')
hog_module = importfile(caminho_modulos+'/descritores/hog.py')
glcm_module = importfile(caminho_modulos+'/descritores/glcm.py')  # NOVO: Módulo GLCM
rf_module = importfile(caminho_modulos+'/classificadores/random_forest.py')
svm_module = importfile(caminho_modulos+'/classificadores/svm.py')
mlp_module = importfile(caminho_modulos+'/classificadores/mlp.py')
knn_module = importfile(caminho_modulos+'/classificadores/knn.py')
dados_utils = importfile(caminho_modulos+'/utils/dados.py')
rotulos_utils = importfile(caminho_modulos+'/utils/rotulos.py')
metricas_utils = importfile(caminho_modulos+'/utils/metricas.py')



########################################################################
# CAMINHOS GLOBAIS
########################################################################

caminho_base = '/content/drive/MyDrive/Colab Notebooks/VC/Extracao_Classificacao'
caminho_todos_datasets = caminho_base + '/datasets/'
selected_dataset = 'covid19'

def configurar_caminhos(caminho_base, dataset, caracteristica):
    """
    Configura variáveis globais com base no caminho base, no nome do dataset e na característica fornecidos.

    Parâmetros:
    - caminho_base (str): O caminho base para o diretório onde os arquivos estão armazenados.
    - dataset (str): O nome do dataset (por exemplo, 'covid19', 'outro_dataset', etc.)
    - caracteristica (str): O nome da característica (por exemplo, 'sift', 'hog', etc.)
    """
    global caminho_dataset_treinamento
    global caminho_dataset_teste
    global caminho_features_treinamento
    global caminho_features_teste
    global caminho_rotulos_treinamento_label
    global caminho_rotulos_teste_label
    global caminho_rotulos_treinamento_onehot
    global caminho_rotulos_teste_onehot
    global caminho_modelo_rf_treinado
    global caminho_modelo_svm_treinado
    global caminho_modelo_mlp_treinado
    global caminho_modelo_knn_treinado
    global caminho_resultado_rf_mc
    global caminho_resultado_rf_rc
    global caminho_resultado_svm_mc
    global caminho_resultado_svm_rc
    global caminho_resultado_mlp_mc
    global caminho_resultado_mlp_rc
    global caminho_resultado_knn_mc
    global caminho_resultado_knn_rc

    # Caminho dos datasets armazenados no Google Drive
    caminho_dataset_treinamento = f'{caminho_base}/datasets/{dataset}/train'
    caminho_dataset_teste = f'{caminho_base}/datasets/{dataset}/test'

    # Caminho das features extraídas armazenadas no Google Drive
    caminho_features_treinamento = f'{caminho_base}/features/{caracteristica}/{dataset}/train/features.npy'
    caminho_features_teste = f'{caminho_base}/features/{caracteristica}/{dataset}/test/features.npy'

    # Caminho dos rótulos extraídos armazenados no Google Drive
    caminho_rotulos_treinamento_label = f'{caminho_base}/features/{caracteristica}/{dataset}/train/rotulos_labelenc.npy'
    caminho_rotulos_teste_label = f'{caminho_base}/features/{caracteristica}/{dataset}/test/rotulos_labelenc.npy'
    caminho_rotulos_treinamento_onehot = f'{caminho_base}/features/{caracteristica}/{dataset}/train/rotulos_onehotenc.npy'
    caminho_rotulos_teste_onehot = f'{caminho_base}/features/{caracteristica}/{dataset}/test/rotulos_onehotenc.npy'

    # Caminho dos modelos treinados armazenados no Google Drive
    caminho_modelo_rf_treinado = f'{caminho_base}/modelos/rf/{dataset}/rf_{caracteristica}.pkl'
    caminho_modelo_svm_treinado = f'{caminho_base}/modelos/svm/{dataset}/svm_{caracteristica}.pkl'
    caminho_modelo_mlp_treinado = f'{caminho_base}/modelos/mlp/{dataset}/mlp_{caracteristica}.pkl'
    caminho_modelo_knn_treinado = f'{caminho_base}/modelos/knn/{dataset}/knn_{caracteristica}.pkl'

    # Caminho dos resultados armazenados no Google Drive
    caminho_resultado_rf_mc = f'{caminho_base}/resultados/rf/rf_{caracteristica}_mc.png'
    caminho_resultado_rf_rc = f'{caminho_base}/resultados/rf/rf_{caracteristica}_rc.png'
    caminho_resultado_svm_mc = f'{caminho_base}/resultados/svm/{dataset}/svm_{caracteristica}_mc.png'
    caminho_resultado_svm_rc = f'{caminho_base}/resultados/svm/{dataset}/svm_{caracteristica}_rc.png'
    caminho_resultado_mlp_mc = f'{caminho_base}/resultados/mlp/{dataset}/mlp_{caracteristica}_mc.png'
    caminho_resultado_mlp_rc = f'{caminho_base}/resultados/mlp/{dataset}/mlp_{caracteristica}_rc.png'
    caminho_resultado_knn_mc = f'{caminho_base}/resultados/knn/{dataset}/knn_{caracteristica}_mc.png'
    caminho_resultado_knn_rc = f'{caminho_base}/resultados/knn/{dataset}/knn_{caracteristica}_rc.png'



# FUNÇÕES CALLBACK ASSOCIADAS AOS ITENS DA INTERFACE DO  USUÁRIO #
##################################################################

def extrair_caracteristicas_treinamento(imagens, rotulos):
    """
    Extrai características de treinamento das imagens fornecidas e salva os resultados em arquivos.

    Parâmetros:
    - imagens (list): Lista de imagens de treinamento.
    - rotulos (list): Lista de rótulos correspondentes às imagens de treinamento.

    Retorno:
    - modelo_kmeans (MiniBatchKMeans): O modelo treinado K-Means.
    - n_grupos (int): Número de grupos (clusters) utilizados no modelo K-Means.
    """

    # Inicializa as variáveis que serão retornadas
    modelo_kmeans = None
    n_grupos = None

    if checkbox_histograma.value:
        # Centralizar o título
        print('\n')
        titulo = f'Extração de Características HISTOGRAMA'
        display(HTML(f'<div style="text-align: left; font-size: 16px; font-weight: bold;">{titulo}</div>'))
        caracteristica = 'histograma'
        configurar_caminhos(caminho_base, selected_dataset, caracteristica)
        # Extrai características histograma e obtém o modelo kmeans treinado
        caracteristicas_histograma_treinamento = histograma_module.extrair_histograma_escala_cinza(imagens)
        # Salva as características extraídas
        dados_utils.salvar_caracteristicas(caracteristicas_histograma_treinamento, caminho_features_treinamento)
        # Codifica os rótulos usando LabelEncoder salva os resultados
        rotulos_label, encoder = rotulos_utils.codificar_rotulos_label(rotulos)
        dados_utils.salvar_rotulos(rotulos_label, encoder, caminho_rotulos_treinamento_label)
        # Codifica os rótulos usando OneHotEncoder salva os resultados
        rotulos_onehot, encoder = rotulos_utils.codificar_rotulos_onehot(rotulos)
        dados_utils.salvar_rotulos(rotulos_onehot, encoder, caminho_rotulos_treinamento_onehot)
    if checkbox_sift.value:
        # Centralizar o título
        print('\n')
        titulo = f'Extração de Características SIFT'
        display(HTML(f'<div style="text-align: left; font-size: 16px; font-weight: bold;">{titulo}</div>'))
        caracteristica = 'sift'
        configurar_caminhos(caminho_base, selected_dataset, caracteristica)
        # Extrai características SIFT e obtém o modelo kmeans treinado
        caracteristicas_sift_treinamento, modelo_kmeans, n_grupos, rotulos_validos = sift_module.extrair_sift_treinamento(imagens,rotulos)
        # Salva as características extraídas
        dados_utils.salvar_caracteristicas(caracteristicas_sift_treinamento, caminho_features_treinamento)
        # Codifica os rótulos usando LabelEncoder salva os resultados
        rotulos_label, encoder = rotulos_utils.codificar_rotulos_label(rotulos_validos)
        dados_utils.salvar_rotulos(rotulos_label, encoder, caminho_rotulos_treinamento_label)
        # Codifica os rótulos usando OneHotEncoder salva os resultados
        rotulos_onehot, encoder = rotulos_utils.codificar_rotulos_onehot(rotulos_validos)
        dados_utils.salvar_rotulos(rotulos_onehot, encoder, caminho_rotulos_treinamento_onehot)
    if checkbox_lbp.value:
        # Centralizar o título
        print('\n')
        titulo = f'Extração de Características LBP'
        display(HTML(f'<div style="text-align: left; font-size: 16px; font-weight: bold;">{titulo}</div>'))
        caracteristica = 'lbp'
        configurar_caminhos(caminho_base, selected_dataset, caracteristica)
        # Extrai características lbp e obtém o modelo kmeans treinado
        caracteristicas_lbp_treinamento = lbp_module.extrair_lbp(imagens)
        # Salva as características extraídas
        dados_utils.salvar_caracteristicas(caracteristicas_lbp_treinamento, caminho_features_treinamento)
        # Codifica os rótulos usando LabelEncoder salva os resultados
        rotulos_label, encoder = rotulos_utils.codificar_rotulos_label(rotulos)
        dados_utils.salvar_rotulos(rotulos_label, encoder, caminho_rotulos_treinamento_label)
        # Codifica os rótulos usando OneHotEncoder salva os resultados
        rotulos_onehot, encoder = rotulos_utils.codificar_rotulos_onehot(rotulos)
        dados_utils.salvar_rotulos(rotulos_onehot, encoder, caminho_rotulos_treinamento_onehot)
    if checkbox_hog.value:
        # Centralizar o título
        print('\n')
        titulo = f'Extração de Características HOG'
        display(HTML(f'<div style="text-align: left; font-size: 16px; font-weight: bold;">{titulo}</div>'))
        caracteristica = 'hog'
        configurar_caminhos(caminho_base, selected_dataset, caracteristica)
        # Extrai características hog e obtém o modelo kmeans treinado
        caracteristicas_hog_treinamento = hog_module.extrair_hog(imagens)
        # Salva as características extraídas
        dados_utils.salvar_caracteristicas(caracteristicas_hog_treinamento, caminho_features_treinamento)
        # Codifica os rótulos usando LabelEncoder salva os resultados
        rotulos_label, encoder = rotulos_utils.codificar_rotulos_label(rotulos)
        dados_utils.salvar_rotulos(rotulos_label, encoder, caminho_rotulos_treinamento_label)
        # Codifica os rótulos usando OneHotEncoder salva os resultados
        rotulos_onehot, encoder = rotulos_utils.codificar_rotulos_onehot(rotulos)
        dados_utils.salvar_rotulos(rotulos_onehot, encoder, caminho_rotulos_treinamento_onehot)
    if checkbox_glcm.value:  # NOVO: Extração GLCM
        # Centralizar o título
        print('\n')
        titulo = f'Extração de Características GLCM'
        display(HTML(f'<div style="text-align: left; font-size: 16px; font-weight: bold;">{titulo}</div>'))
        caracteristica = 'glcm'
        configurar_caminhos(caminho_base, selected_dataset, caracteristica)
        # Extrai características GLCM
        caracteristicas_glcm_treinamento = glcm_module.extrair_glcm(imagens)
        # Salva as características extraídas
        dados_utils.salvar_caracteristicas(caracteristicas_glcm_treinamento, caminho_features_treinamento)
        # Codifica os rótulos usando LabelEncoder salva os resultados
        rotulos_label, encoder = rotulos_utils.codificar_rotulos_label(rotulos)
        dados_utils.salvar_rotulos(rotulos_label, encoder, caminho_rotulos_treinamento_label)
        # Codifica os rótulos usando OneHotEncoder salva os resultados
        rotulos_onehot, encoder = rotulos_utils.codificar_rotulos_onehot(rotulos)
        dados_utils.salvar_rotulos(rotulos_onehot, encoder, caminho_rotulos_treinamento_onehot)

    return modelo_kmeans, n_grupos


def extrair_caracteristicas_teste(imagens, rotulos, modelo_kmeans, n_grupos):
    """
    Extrai características de teste das imagens fornecidas e salva os resultados em arquivos.

    Parâmetros:
    - imagens (list): Lista de imagens de teste.
    - rotulos (list): Lista de rótulos correspondentes às imagens de teste.
    - modelo_kmeans (MiniBatchKMeans): O modelo K-Means treinado.
    - n_grupos (int): Número de grupos (clusters) utilizados no modelo K-Means.

    Retorno:
    - None
    """
    if checkbox_histograma.value:
        caracteristica = 'histograma'
        configurar_caminhos(caminho_base, selected_dataset, caracteristica)
        # Extrai características histograma e obtém o modelo kmeans treinado
        caracteristicas_histograma_teste = histograma_module.extrair_histograma_escala_cinza(imagens)
        # Salva as características extraídas
        dados_utils.salvar_caracteristicas(caracteristicas_histograma_teste, caminho_features_teste)
        # Codifica os rótulos usando LabelEncoder salva os resultados
        rotulos_label, encoder = rotulos_utils.codificar_rotulos_label(rotulos)
        dados_utils.salvar_rotulos(rotulos_label, encoder, caminho_rotulos_teste_label)
        # Codifica os rótulos usando OneHotEncoder salva os resultados
        rotulos_onehot, encoder = rotulos_utils.codificar_rotulos_onehot(rotulos)
        dados_utils.salvar_rotulos(rotulos_onehot, encoder, caminho_rotulos_teste_onehot)
    if checkbox_sift.value:
        caracteristica = 'sift'
        configurar_caminhos(caminho_base, selected_dataset, caracteristica)
        # Extrai características SIFT e gera histogramas BoVW para o conjunto de teste
        caracteristicas_sift_teste, rotulos_validos = sift_module.extrair_sift_teste(imagens, modelo_kmeans, n_grupos, rotulos)
        dados_utils.salvar_caracteristicas(caracteristicas_sift_teste, caminho_features_teste)
        # Codifica os rótulos usando LabelEncoder salva os resultados
        rotulos_label, encoder = rotulos_utils.codificar_rotulos_label(rotulos_validos)
        dados_utils.salvar_rotulos(rotulos_label, encoder, caminho_rotulos_teste_label)
        # Codifica os rótulos usando OneHotEncoder salva os resultados
        rotulos_onehot, encoder = rotulos_utils.codificar_rotulos_onehot(rotulos_validos)
        dados_utils.salvar_rotulos(rotulos_onehot, encoder, caminho_rotulos_teste_onehot)
    if checkbox_lbp.value:
        caracteristica = 'lbp'
        configurar_caminhos(caminho_base, selected_dataset, caracteristica)
        # Extrai características lbp e obtém o modelo kmeans treinado
        caracteristicas_lbp_teste = lbp_module.extrair_lbp(imagens)
        # Salva as características extraídas
        dados_utils.salvar_caracteristicas(caracteristicas_lbp_teste, caminho_features_teste)
        # Codifica os rótulos usando LabelEncoder salva os resultados
        rotulos_label, encoder = rotulos_utils.codificar_rotulos_label(rotulos)
        dados_utils.salvar_rotulos(rotulos_label, encoder, caminho_rotulos_teste_label)
        # Codifica os rótulos usando OneHotEncoder salva os resultados
        rotulos_onehot, encoder = rotulos_utils.codificar_rotulos_onehot(rotulos)
        dados_utils.salvar_rotulos(rotulos_onehot, encoder, caminho_rotulos_teste_onehot)
    if checkbox_hog.value:
        caracteristica = 'hog'
        configurar_caminhos(caminho_base, selected_dataset, caracteristica)
        # Extrai características hog e obtém o modelo kmeans treinado
        caracteristicas_hog_teste = hog_module.extrair_hog(imagens)
        # Salva as características extraídas
        dados_utils.salvar_caracteristicas(caracteristicas_hog_teste, caminho_features_teste)
        # Codifica os rótulos usando LabelEncoder salva os resultados
        rotulos_label, encoder = rotulos_utils.codificar_rotulos_label(rotulos)
        dados_utils.salvar_rotulos(rotulos_label, encoder, caminho_rotulos_teste_label)
        # Codifica os rótulos usando OneHotEncoder salva os resultados
        rotulos_onehot, encoder = rotulos_utils.codificar_rotulos_onehot(rotulos)
        dados_utils.salvar_rotulos(rotulos_onehot, encoder, caminho_rotulos_teste_onehot)
    if checkbox_glcm.value:  # NOVO: Extração GLCM para teste
        caracteristica = 'glcm'
        configurar_caminhos(caminho_base, selected_dataset, caracteristica)
        # Extrai características GLCM
        caracteristicas_glcm_teste = glcm_module.extrair_glcm(imagens)
        # Salva as características extraídas
        dados_utils.salvar_caracteristicas(caracteristicas_glcm_teste, caminho_features_teste)
        # Codifica os rótulos usando LabelEncoder salva os resultados
        rotulos_label, encoder = rotulos_utils.codificar_rotulos_label(rotulos)
        dados_utils.salvar_rotulos(rotulos_label, encoder, caminho_rotulos_teste_label)
        # Codifica os rótulos usando OneHotEncoder salva os resultados
        rotulos_onehot, encoder = rotulos_utils.codificar_rotulos_onehot(rotulos)
        dados_utils.salvar_rotulos(rotulos_onehot, encoder, caminho_rotulos_teste_onehot)



def carregar_imagens_conjuntos(tipo_conjunto):
    """
    Carrega imagens e rótulos para o conjunto especificado ('treinamento' ou 'teste').

    Parâmetros:
    - tipo_conjunto (str): O tipo de conjunto a ser carregado ('treinamento' ou 'teste').

    Retorno:
    - imagens (list): Lista de imagens carregadas.
    - rotulos (list): Lista de rótulos correspondentes às imagens.
    """
    if tipo_conjunto == 'treinamento':
        # Centralizar o título
        titulo = f'CONJUNTO DE TREINAMENTO'
        display(HTML(f'<div style="text-align: center; font-size: 20px; font-weight: bold;">{titulo}</div>'))
        print('\n')
        configurar_caminhos(caminho_base, selected_dataset, None)
        # Carrega imagens e rótulos de treinamento
        imagens, rotulos = dados_utils.carregar_imagens(caminho_dataset_treinamento)
    elif tipo_conjunto == 'teste':
        # Centralizar o título
        print('\n')
        titulo = f'CONJUNTO DE TESTE'
        display(HTML(f'<div style="text-align: center; font-size: 20px; font-weight: bold;">{titulo}</div>'))
        print('\n')
        configurar_caminhos(caminho_base, selected_dataset, None)
        # Carrega imagens e rótulos de teste
        imagens, rotulos = dados_utils.carregar_imagens(caminho_dataset_teste)
    else:
        raise ValueError(f"Tipo de conjunto {tipo_conjunto} não é válido.")

    # Retorna as imagens e os rótulos correspondentes ao tipo de conjunto
    return imagens, rotulos


def extrair_caracteristicas_conjuntos():
    """
    Extrai características para os conjuntos de treinamento e teste, e salva os resultados.
    """
    limpar_outputs()
    with main_output:  # Usa o widget de saída para capturar toda a saída gerada
        # Carrega e processa o conjunto de treinamento
        imagens, rotulos = carregar_imagens_conjuntos('treinamento')
        modelo_kmeans, n_grupos = extrair_caracteristicas_treinamento(imagens, rotulos)

        # Carrega e processa o conjunto de teste
        imagens, rotulos_teste = carregar_imagens_conjuntos('teste')
        extrair_caracteristicas_teste(imagens, rotulos_teste, modelo_kmeans, n_grupos)
        display(HTML('<div style="text-align: center; color: green; padding: 10px;">Características Extraídas com Sucesso!</div>'))


def treinar_modelo(classificador, caracteristica, caminho_features_treinamento, caminho_rotulos_treinamento_label, caminho_modelo_treinado):
    """
    Treina um modelo com base no classificador e característica fornecidos.

    Parâmetros:
    - classificador (str): O classificador a ser utilizado ('rf', 'svm', 'mlp', 'knn').
    - caracteristica (str): A característica a ser utilizada ('sift', 'hog', etc.).
    - caminho_features_treinamento (str): Caminho para as características de treinamento.
    - caminho_rotulos_treinamento_label (str): Caminho para os rótulos de treinamento.
    - caminho_modelo_treinado (str): Caminho para salvar o modelo treinado.
    """
    print('\n')
    # Centralizar o título
    titulo = f'Treinamento {classificador.upper()} - {caracteristica.upper()}'
    display(HTML(f'<div style="text-align: center; font-size: 20px; font-weight: bold;">{titulo}</div>'))

    # Carregar características e rótulos
    caracteristicas = dados_utils.carregar_caracteristicas(caminho_features_treinamento)
    rotulos, encoder = dados_utils.carregar_rotulos(caminho_rotulos_treinamento_label)

    # Treinamento do modelo baseado no classificador
    if classificador == 'rf':
        modelo = rf_module.treinar_rf(caracteristicas, rotulos)
    elif classificador == 'svm':
        modelo = svm_module.treinar_svm(caracteristicas, rotulos)
    elif classificador == 'mlp':
        modelo = mlp_module.treinar_mlp(caracteristicas, rotulos)
    elif classificador == 'knn':
        modelo = knn_module.treinar_knn(caracteristicas, rotulos)
    else:
        raise ValueError("Classificador inválido. Escolha entre 'rf', 'svm', 'mlp', 'knn'.")

    # Salvar o modelo treinado
    dados_utils.salvar_modelo(modelo, caminho_modelo_treinado)

def treinar_classificadores():
    limpar_outputs()
    with main_output:  # Usa o widget de saída para capturar toda a saída gerada
        if checkbox_rf_train.value:
            if dropdown_features_train.value == 'Histograma':
                caracteristica = 'histograma'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('rf', 'histograma', caminho_features_treinamento, caminho_rotulos_treinamento_label, caminho_modelo_rf_treinado)
            elif dropdown_features_train.value == 'SIFT':
                caracteristica = 'sift'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('rf', 'sift', caminho_features_treinamento, caminho_rotulos_treinamento_label, caminho_modelo_rf_treinado)
            elif dropdown_features_train.value == 'LBP':
                caracteristica = 'lbp'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('rf', 'histograma', caminho_features_treinamento, caminho_rotulos_treinamento_label, caminho_modelo_rf_treinado)
            elif dropdown_features_train.value == 'HOG':
                caracteristica = 'hog'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('rf', 'hog', caminho_features_treinamento, caminho_rotulos_treinamento_label, caminho_modelo_rf_treinado)
            elif dropdown_features_train.value == 'GLCM':  # NOVO: Treinamento com GLCM
                caracteristica = 'glcm'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('rf', 'glcm', caminho_features_treinamento, caminho_rotulos_treinamento_label, caminho_modelo_rf_treinado)
        if checkbox_svm_train.value:
            if dropdown_features_train.value == 'Histograma':
                caracteristica = 'histograma'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('svm', 'histograma', caminho_features_treinamento, caminho_rotulos_treinamento_label, caminho_modelo_svm_treinado)
            elif dropdown_features_train.value == 'SIFT':
                caracteristica = 'sift'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('svm', 'sift', caminho_features_treinamento, caminho_rotulos_treinamento_label, caminho_modelo_svm_treinado)
            elif dropdown_features_train.value == 'LBP':
                caracteristica = 'lbp'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('svm', 'lbp', caminho_features_treinamento, caminho_rotulos_treinamento_label, caminho_modelo_svm_treinado)
            elif dropdown_features_train.value == 'HOG':
                caracteristica = 'hog'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('svm', 'hog', caminho_features_treinamento, caminho_rotulos_treinamento_label, caminho_modelo_svm_treinado)
            elif dropdown_features_train.value == 'GLCM':  # NOVO: Treinamento SVM com GLCM
                caracteristica = 'glcm'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('svm', 'glcm', caminho_features_treinamento, caminho_rotulos_treinamento_label, caminho_modelo_svm_treinado)
        if checkbox_knn_train.value:
            if dropdown_features_train.value == 'Histograma':
                caracteristica = 'histograma'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('knn', 'histograma', caminho_features_treinamento, caminho_rotulos_treinamento_label, caminho_modelo_knn_treinado)
            elif dropdown_features_train.value == 'SIFT':
                caracteristica = 'sift'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('knn', 'sift', caminho_features_treinamento, caminho_rotulos_treinamento_label, caminho_modelo_knn_treinado)
            elif dropdown_features_train.value == 'LBP':
                caracteristica = 'lbp'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('knn', 'lbp', caminho_features_treinamento, caminho_rotulos_treinamento_label, caminho_modelo_knn_treinado)
            elif dropdown_features_train.value == 'HOG':
                caracteristica = 'hog'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('knn', 'hog', caminho_features_treinamento, caminho_rotulos_treinamento_label, caminho_modelo_knn_treinado)
            elif dropdown_features_train.value == 'GLCM':  # NOVO: Treinamento KNN com GLCM
                caracteristica = 'glcm'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('knn', 'glcm', caminho_features_treinamento, caminho_rotulos_treinamento_label, caminho_modelo_knn_treinado)
        if checkbox_mlp_train.value:
            if dropdown_features_train.value == 'Histograma':
                caracteristica = 'histograma'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('mlp', 'histograma', caminho_features_treinamento, caminho_rotulos_treinamento_onehot, caminho_modelo_mlp_treinado)
            elif dropdown_features_train.value == 'SIFT':
                caracteristica = 'sift'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('mlp', 'sift', caminho_features_treinamento, caminho_rotulos_treinamento_onehot, caminho_modelo_mlp_treinado)
            elif dropdown_features_train.value == 'LBP':
                caracteristica = 'lbp'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('mlp', 'lbp', caminho_features_treinamento, caminho_rotulos_treinamento_onehot, caminho_modelo_mlp_treinado)
            elif dropdown_features_train.value == 'HOG':
                caracteristica = 'hog'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('mlp', 'hog', caminho_features_treinamento, caminho_rotulos_treinamento_onehot, caminho_modelo_mlp_treinado)
            elif dropdown_features_train.value == 'GLCM':  # NOVO: Treinamento MLP com GLCM
                caracteristica = 'glcm'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                treinar_modelo('mlp', 'glcm', caminho_features_treinamento, caminho_rotulos_treinamento_onehot, caminho_modelo_mlp_treinado)
        display(HTML('<div style="text-align: center; color: green; padding: 10px;">Modelo (s) Treinado (s) com Sucesso!</div>'))


def testar_modelo(classificador, caracteristica, caminho_features_teste, caminho_rotulos_teste, caminho_modelo_treinado, caminho_resultado_mc, caminho_resultado_rc):
    """
    Função para testar o modelo com base no classificador e característica fornecidos.

    :param classificador: Tipo de classificador ('rf', 'svm', 'mlp', 'knn')
    :param caracteristica: Tipo de característica ('sift', 'histograma', etc.)
    :param caminho_features_teste: Caminho para as características de teste
    :param caminho_rotulos_teste: Caminho para os rótulos de teste
    :param caminho_modelo_treinado: Caminho para o modelo treinado
    :param caminho_resultado_mc: Caminho para salvar a matriz de confusão
    :param caminho_resultado_rc: Caminho para salvar o relatório de classificação
    """
    print('\n')
    display(HTML(f'<div style="text-align: center; font-size: 20px; font-weight: bold;">Classificação - {classificador.upper()} - {caracteristica.upper()}</div>'))

    # Carregar características
    caracteristicas = dados_utils.carregar_caracteristicas(caminho_features_teste)

    # Escolher o tipo de codificação dos rótulos
    rotulos_verdadeiros, encoder = dados_utils.carregar_rotulos(caminho_rotulos_teste)

    # Carregar o modelo treinado
    try:
        modelo = dados_utils.carregar_modelo(caminho_modelo_treinado)
    except FileNotFoundError:
        print(f"Modelo não encontrado no caminho: {caminho_modelo_treinado}")
        modelo = None
    #  Fazer a previsão
    if modelo is not None:
        if classificador == 'rf':
            rotulos_previstos = rf_module.testar_rf(modelo, caracteristicas)
        elif classificador == 'svm':
            rotulos_previstos = svm_module.testar_svm(modelo, caracteristicas)
        elif classificador == 'mlp':
            rotulos_previstos = mlp_module.testar_mlp(modelo, caracteristicas)
            # Decodificar os rótulos previstos e verdadeiros de volta para a forma original
            rotulos_previstos_ajustados = mlp_module.ajustar_amostras_zero(rotulos_previstos)
            rotulos_previstos = encoder.inverse_transform(rotulos_previstos_ajustados)
            rotulos_verdadeiros = encoder.inverse_transform(rotulos_verdadeiros)
        elif classificador == 'knn':
            rotulos_previstos = knn_module.testar_knn(modelo, caracteristicas)
        else:
            raise ValueError(f"Classificador {classificador} não suportado.")

        # Obter os nomes das classes
        if isinstance(encoder, preprocessing.OneHotEncoder):
            # Se for OneHotEncoder, obtenha as classes diretamente do rótulo original
            nomes_das_classes = encoder.categories_[0]
        else:
            # Se for LabelEncoder, use o atributo classes_
            nomes_das_classes = encoder.classes_

        # Gerar e salvar as métricas
        print('\n')
        metricas_utils.matriz_confusao(nomes_das_classes, rotulos_verdadeiros, rotulos_previstos, caminho_resultado_mc)
        print('\n')
        metricas_utils.relatorio_classificacao(nomes_das_classes, rotulos_verdadeiros, rotulos_previstos, caminho_resultado_rc)



def testar_classificadores():
    limpar_outputs()
    with main_output:  # Usa o widget de saída para capturar toda a saída gerada
        if checkbox_rf_test.value:
            if dropdown_features_test.value == 'Histograma':
                caracteristica = 'histograma'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'rf',
                    'histograma',
                    caminho_features_teste,
                    caminho_rotulos_teste_label,
                    caminho_modelo_rf_treinado,
                    caminho_resultado_rf_mc,
                    caminho_resultado_rf_rc
                )
            elif dropdown_features_test.value == 'SIFT':
                caracteristica = 'sift'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'rf',
                    'sift',
                    caminho_features_teste,
                    caminho_rotulos_teste_label,
                    caminho_modelo_rf_treinado,
                    caminho_resultado_rf_mc,
                    caminho_resultado_rf_rc
                )
            elif dropdown_features_test.value == 'LBP':
                caracteristica = 'lbp'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'rf',
                    'lbp',
                    caminho_features_teste,
                    caminho_rotulos_teste_label,
                    caminho_modelo_rf_treinado,
                    caminho_resultado_rf_mc,
                    caminho_resultado_rf_rc
                )
            elif dropdown_features_test.value == 'HOG':
                caracteristica = 'hog'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'rf',
                    'hog',
                    caminho_features_teste,
                    caminho_rotulos_teste_label,
                    caminho_modelo_rf_treinado,
                    caminho_resultado_rf_mc,
                    caminho_resultado_rf_rc
                )
            elif dropdown_features_test.value == 'GLCM':  # NOVO: Teste RF com GLCM
                caracteristica = 'glcm'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'rf',
                    'glcm',
                    caminho_features_teste,
                    caminho_rotulos_teste_label,
                    caminho_modelo_rf_treinado,
                    caminho_resultado_rf_mc,
                    caminho_resultado_rf_rc
                )
        if checkbox_svm_test.value:
            if dropdown_features_test.value == 'Histograma':
                caracteristica = 'histograma'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'svm',
                    'histograma',
                    caminho_features_teste,
                    caminho_rotulos_teste_label,
                    caminho_modelo_svm_treinado,
                    caminho_resultado_svm_mc,
                    caminho_resultado_svm_rc
                )
            elif dropdown_features_test.value == 'SIFT':
                caracteristica = 'sift'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'svm',
                    'sift',
                    caminho_features_teste,
                    caminho_rotulos_teste_label,
                    caminho_modelo_svm_treinado,
                    caminho_resultado_svm_mc,
                    caminho_resultado_svm_rc
                )
            elif dropdown_features_test.value == 'LBP':
                caracteristica = 'lbp'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'svm',
                    'lbp',
                    caminho_features_teste,
                    caminho_rotulos_teste_label,
                    caminho_modelo_svm_treinado,
                    caminho_resultado_svm_mc,
                    caminho_resultado_svm_rc
                )
            elif dropdown_features_test.value == 'HOG':
                caracteristica = 'hog'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'svm',
                    'hog',
                    caminho_features_teste,
                    caminho_rotulos_teste_label,
                    caminho_modelo_svm_treinado,
                    caminho_resultado_svm_mc,
                    caminho_resultado_svm_rc
                )
            elif dropdown_features_test.value == 'GLCM':  # NOVO: Teste SVM com GLCM
                caracteristica = 'glcm'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'svm',
                    'glcm',
                    caminho_features_teste,
                    caminho_rotulos_teste_label,
                    caminho_modelo_svm_treinado,
                    caminho_resultado_svm_mc,
                    caminho_resultado_svm_rc
                )
        if checkbox_knn_test.value:
            if dropdown_features_test.value == 'Histograma':
                caracteristica = 'histograma'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'knn',
                    'histograma',
                    caminho_features_teste,
                    caminho_rotulos_teste_label,
                    caminho_modelo_knn_treinado,
                    caminho_resultado_knn_mc,
                    caminho_resultado_knn_rc
                )
            elif dropdown_features_test.value == 'SIFT':
                caracteristica = 'sift'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'knn',
                    'sift',
                    caminho_features_teste,
                    caminho_rotulos_teste_label,
                    caminho_modelo_knn_treinado,
                    caminho_resultado_knn_mc,
                    caminho_resultado_knn_rc
                )
            elif dropdown_features_test.value == 'LBP':
                caracteristica = 'lbp'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'knn',
                    'lbp',
                    caminho_features_teste,
                    caminho_rotulos_teste_label,
                    caminho_modelo_knn_treinado,
                    caminho_resultado_knn_mc,
                    caminho_resultado_knn_rc
                )
            elif dropdown_features_test.value == 'HOG':
                caracteristica = 'hog'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'knn',
                    'hog',
                    caminho_features_teste,
                    caminho_rotulos_teste_label,
                    caminho_modelo_knn_treinado,
                    caminho_resultado_knn_mc,
                    caminho_resultado_knn_rc
                )
            elif dropdown_features_test.value == 'GLCM':  # NOVO: Teste KNN com GLCM
                caracteristica = 'glcm'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'knn',
                    'glcm',
                    caminho_features_teste,
                    caminho_rotulos_teste_label,
                    caminho_modelo_knn_treinado,
                    caminho_resultado_knn_mc,
                    caminho_resultado_knn_rc
                )
        if checkbox_mlp_test.value:
            if dropdown_features_test.value == 'Histograma':
                caracteristica = 'histograma'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'mlp',
                    'histograma',
                    caminho_features_teste,
                    caminho_rotulos_teste_onehot,
                    caminho_modelo_mlp_treinado,
                    caminho_resultado_mlp_mc,
                    caminho_resultado_mlp_rc
                )
            elif dropdown_features_test.value == 'SIFT':
                caracteristica = 'sift'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'mlp',
                    'sift',
                    caminho_features_teste,
                    caminho_rotulos_teste_onehot,
                    caminho_modelo_mlp_treinado,
                    caminho_resultado_mlp_mc,
                    caminho_resultado_mlp_rc
                )
            elif dropdown_features_test.value == 'LBP':
                caracteristica = 'lbp'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'mlp',
                    'lbp',
                    caminho_features_teste,
                    caminho_rotulos_teste_onehot,
                    caminho_modelo_mlp_treinado,
                    caminho_resultado_mlp_mc,
                    caminho_resultado_mlp_rc
                )
            elif dropdown_features_test.value == 'HOG':
                caracteristica = 'hog'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'mlp',
                    'hog',
                    caminho_features_teste,
                    caminho_rotulos_teste_onehot,
                    caminho_modelo_mlp_treinado,
                    caminho_resultado_mlp_mc,
                    caminho_resultado_mlp_rc
                )
            elif dropdown_features_test.value == 'GLCM':  # NOVO: Teste MLP com GLCM
                caracteristica = 'glcm'
                configurar_caminhos(caminho_base, selected_dataset, caracteristica)
                testar_modelo(
                    'mlp',
                    'glcm',
                    caminho_features_teste,
                    caminho_rotulos_teste_onehot,
                    caminho_modelo_mlp_treinado,
                    caminho_resultado_mlp_mc,
                    caminho_resultado_mlp_rc
                )
        display(HTML('<div style="text-align: center; color: green; padding: 10px;">Teste (s) encerrado (s) !</div>'))





#################################
# ITENS DA INTERFACE DO USUÁRIO #
#################################

#############
# CABEÇALHO #
#############

# Header
header = widgets.HTML('<font color="white" face="sans-serif">'+
                      '<center><h1>Extração de Características e Classificação<br>Abordagem Clássica</h1></center>'+
                      '</font>',
                      layout=widgets.Layout(height='auto'))

###################
# ABAS PRINCIPAIS #
###################

# Criar abas
tab = widgets.Tab(
    layout=widgets.Layout(min_width='700px', min_height='50px') # Definir tamanho mínimo
)

# Definir os títulos de cada aba
tab.set_title(0, 'DATASET')
tab.set_title(1, 'EXT. CARACTERÍSTICAS')
tab.set_title(2, 'TREINAMENTO')
tab.set_title(3, 'CLASSIFICAÇÃO')

# Output widget para exibir os resultados
main_output = widgets.Output(
    layout=widgets.Layout(
        border='0px solid black',
        width='100%',
        height='60vh',  # Define a altura como 50% da altura da tela
        overflow_y='auto'  # Adiciona barra de rolagem vertical
    )
)


def limpar_outputs():
    main_output.clear_output(wait=True)


# HBox para organizar image_output, hist_output e segmentation lado a lado
outputs_box = widgets.VBox([main_output], layout=widgets.Layout(align_items='center', justify_content='center'))

# GridBox para organizar o cabeçalho, as abas e a imagem de saída
grid = widgets.GridBox(children=[header, tab, outputs_box],
               layout=widgets.Layout(
                   grid_template_rows='auto auto auto',
                   grid_template_columns='1fr',  #uma  coluna flexível para ocupar o espaço disponível.
                   justify_items='center',
                   align_items='center',
                   grid_gap='10px'
               ))

# Contêiner pai para centralizar o GridBox
container = widgets.Box([grid], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))


# Exibir o contêiner pai
display(container)


##########################
# CONTEUDO ABA DATASET   #
##########################

# Função de callback que será chamada quando o valor do dropdown mudar
def on_dataset_change(change):
    global selected_dataset
    selected_dataset = change['new']
    #print(f'Dataset selecionado: {selected_dataset}')


def atualizar_dropdown_datasets(caminho_dos_datasets):
    """
    Atualiza o dropdown com os nomes das pastas encontradas no caminho_base.

    :param caminho_base: Caminho base onde as pastas de datasets estão localizadas.
    :return: Um widget Dropdown populado com os nomes das pastas.
    """
    caminho_todos_datasets = os.path.join(caminho_base, 'datasets')

    # Lista as pastas dentro do caminho_todos_datasets
    nomes_datasets = [nome for nome in os.listdir(caminho_todos_datasets) if os.path.isdir(os.path.join(caminho_todos_datasets, nome))]

    # Cria a lista de opções para o dropdown (nome legível, valor)
    options = [(nome.replace('_', ' ').upper(), nome) for nome in nomes_datasets]

    # Cria e retorna o dropdown
    dropdown_datasets = widgets.Dropdown(
        options=options,
        description='Dataset:',
    )

    return dropdown_datasets

# Dropdown para selecionar um dataset
dropdown_datasets = atualizar_dropdown_datasets(caminho_todos_datasets)

# Conectar a função de callback ao evento de alteração do dropdown
dropdown_datasets.observe(on_dataset_change, names='value')

#############################
# CONTEUDO ABA EXT. CARACT. #
#############################

# Criando os checkboxes
checkbox_histograma = widgets.Checkbox(value=False, description='Histograma')
checkbox_lbp = widgets.Checkbox(value=False, description='LBP')
checkbox_sift = widgets.Checkbox(value=False, description='SIFT')
checkbox_hog = widgets.Checkbox(value=False, description='HOG')
checkbox_glcm = widgets.Checkbox(value=False, description='GLCM')  # NOVO: Checkbox para GLCM

# Botão
botao_extrair = widgets.Button(description='Extrair Características')
# Função para ser chamada ao clicar no botão
def on_extrair_click(b):
    extrair_caracteristicas_conjuntos()

# Atribuir função ao botão
botao_extrair.on_click(on_extrair_click)

opcoes_descritores_1 = widgets.HBox([checkbox_histograma, checkbox_lbp, checkbox_glcm],  # NOVO: Adicionado GLCM
                                  layout=widgets.Layout(align_items='center', justify_content='center'))

opcoes_descritores_2 = widgets.HBox([checkbox_sift, checkbox_hog],
                                  layout=widgets.Layout(align_items='center', justify_content='center'))

box_botao_extrair = widgets.HBox([botao_extrair],
                                  layout=widgets.Layout(align_items='center', justify_content='center'))


opcoes_descritores = widgets.VBox([opcoes_descritores_1, opcoes_descritores_2,box_botao_extrair])


#############################
# CONTEUDO ABA TREINAMENTO  #
#############################

# Criando os checkboxes
checkbox_mlp_train = widgets.Checkbox(value=False, description='MLP')
checkbox_rf_train = widgets.Checkbox(value=False, description='RF')
checkbox_svm_train = widgets.Checkbox(value=False, description='SVM')
checkbox_knn_train = widgets.Checkbox(value=False, description='KNN')

# Dropdown para selecionar as características
dropdown_features_train = widgets.Dropdown(
    options=['Histograma','SIFT','LBP','HOG','GLCM'],  # NOVO: Adicionado GLCM
    description='Features:',
)

# Botão
botao_treinar = widgets.Button(description='Treinar')
# Função para ser chamada ao clicar no botão
def on_treinar_click(b):
    treinar_classificadores()

# Atribuir função ao botão
botao_treinar.on_click(on_treinar_click)

box_train_1 = widgets.VBox([checkbox_mlp_train, checkbox_rf_train],
                                  layout=widgets.Layout(align_items='center', justify_content='center'))

box_train_2 = widgets.VBox([checkbox_svm_train, checkbox_knn_train],
                                  layout=widgets.Layout(align_items='center', justify_content='center'))

box_train_3 = widgets.VBox([dropdown_features_train],
                                  layout=widgets.Layout(align_items='center', justify_content='center'))

box_classif_features_train = widgets.HBox([box_train_1,
                                                box_train_2,box_train_3],
                                  layout=widgets.Layout(align_items='center', justify_content='center'))


box_botao_treinar = widgets.HBox([botao_treinar],
                                  layout=widgets.Layout(align_items='center', justify_content='center'))



opcoes_classif_treino = widgets.VBox([box_classif_features_train,
                                    box_botao_treinar])

#############################
# CONTEUDO ABA CLASSIFICACAO  #
#############################

# Criando os checkboxes
checkbox_mlp_test = widgets.Checkbox(value=False, description='MLP')
checkbox_rf_test = widgets.Checkbox(value=False, description='RF')
checkbox_svm_test = widgets.Checkbox(value=False, description='SVM')
checkbox_knn_test = widgets.Checkbox(value=False, description='KNN')


# Dropdown para selecionar as características
dropdown_features_test = widgets.Dropdown(
    options=['Histograma','SIFT','LBP','HOG','GLCM'],  # NOVO: Adicionado GLCM
    description='Features:',
)

# Botão
botao_treinar = widgets.Button(description='Classificar')
# Função para ser chamada ao clicar no botão
def on_treinar_click(b):
    testar_classificadores()

# Atribuir função ao botão
botao_treinar.on_click(on_treinar_click)

box_test_1 = widgets.VBox([checkbox_mlp_test, checkbox_rf_test],
                                  layout=widgets.Layout(align_items='center', justify_content='center'))

box_test_2 = widgets.VBox([checkbox_svm_test, checkbox_knn_test],
                                  layout=widgets.Layout(align_items='center', justify_content='center'))

box_test_3 = widgets.VBox([dropdown_features_test],
                                  layout=widgets.Layout(align_items='center', justify_content='center'))

box_classif_features_test = widgets.HBox([box_test_1,
                                                box_test_2,box_test_3],
                                  layout=widgets.Layout(align_items='center', justify_content='center'))


box_botao_treinar = widgets.HBox([botao_treinar],
                                  layout=widgets.Layout(align_items='center', justify_content='center'))



opcoes_classif_teste = widgets.VBox([box_classif_features_test,
                                    box_botao_treinar])

# Definir o conteúdo de cada subaba
tab.children = [
    widgets.HBox([dropdown_datasets],
                 layout=widgets.Layout(align_items='center', justify_content='center')),
    widgets.HBox([opcoes_descritores],
                 layout=widgets.Layout(align_items='center', justify_content='center')),
    widgets.HBox([opcoes_classif_treino],
                 layout=widgets.Layout(align_items='center', justify_content='center')),
    widgets.HBox([opcoes_classif_teste],
                 layout=widgets.Layout(align_items='center', justify_content='center'))
]




################ FIM DO SCRIPT ########################


ModuleNotFoundError: No module named 'ipywidgets'